In [372]:
library(readxl)
library(data.table)
library(ggplot2)
library(ggprism)
library(ggforce)
library(ggetho)
library(ggsignif)
library(zoo)
theme_set(theme_prism(base_size=20))
options(repr.plot.width=15, repr.plot.height=15)
source("R/plot.R")
source("R/Cbind.R")
source("R/link_idoc_metadata.R")
source("R/library.R")

pipeline_parameters <- yaml::read_yaml("parameters.yaml")
# whether to use the user annotated result in the metadata files (which could be biased)
# or exclusively the computer generated result (which will very little or no bias)
SCORING_SYSTEM <- pipeline_parameters$SCORING_SYSTEM
# whether to use flies where at least one of the trials has an undetermined preference 
# in that case, the final value is the mean of the trials with a determined preference
# example: if the result of two trials is -0.3 and NA, the final result is -0.3 (not -0.15)
USE_INCOMPLETE_TESTS <- pipeline_parameters$USE_INCOMPLETE_TESTS

In [373]:
metadata <- data.table::fread("metadata.csv")

In [374]:
metadata[, sort(unique(Files))]

[1] "2021-09-30 (CH)"                             
  [2] "2021-09-30 (CH1)"                            
  [3] "2021-10-01 (CH)"                             
  [4] "2021-10-01 (CH1)"                            
  [5] "2021-10-01 (CH2)"                            
  [6] "2021-10-02 (CH)"                             
  [7] "2021-10-02 (CH1)"                            
  [8] "2021-10-03 (CH)"                             
  [9] "2021-10-03 (CH1)"                            
 [10] "2021-10-03 (CH2)"                            
 [11] "2021-10-04 (CH)"                             
 [12] "2021-10-04 (CH1)"                            
 [13] "2021-10-05 (CH)"                             
 [14] "2021-10-06 (CH)"                             
 [15] "2021-10-06 (CH1)"                            
 [16] "2021-10-27 (CH)"                             
 [17] "2021-10-27 (CH1)"                            
 [18] "2021-10-29 (CH)"                             
 [19] "2021-11-01 (CH)"                             
 [20] "2021-11-01 (CH1)"                            
 [21] "2021-11-03 (CH1)"                            
 [22] "2021-11-04 (CH)"                             
 [23] "2021-11-08 (CH)"                             
 [24] "2021-11-17 (CH)"                             
 [25] "2021-11-17 (CH1)"                            
 [26] "2021-11-24 (CH)"                             
 [27] "2021-11-27 (CH1)"                            
 [28] "2021-11-28 (CH)"                             
 [29] "2021-11-30 (CH)"                             
 [30] "2021-12-02 (CH)"                             
 [31] "2021-12-03 (CH)"                             
 [32] "2021-12-05 (CH)"                             
 [33] "2021-12-08 (CH)"                             
 [34] "2021-12-17 (CH)"                             
 [35] "2021-12-22 (CH)"                             
 [36] "2021-12-22 (CH1)"                            
 [37] "2021-12-28 (CH)"                             
 [38] "2022-01-03 (CH)"                             
 [39] "2022-06-23 (CH)"                             
 [40] "2022-06-23 (CH1)"                            
 [41] "2023-06-13_15-41-34(ESB-GTACR2)"             
 [42] "2023-06-22_(ESB-GTACR2)"                     
 [43] "2023-06-22_15-37-53(ESB-GTACR2)"             
 [44] "2023-06-27_16-34-26(ET_3h)"                  
 [45] "2023-06-28_15-17-27(ET_3h)"                  
 [46] "2023-07-04_13-45-39(ET_3h)"                  
 [47] "2023-07-10_15-09-51(ET_3h)"                  
 [48] "2023-07-10_16-10-48(ET_1h)"                  
 [49] "2023-07-11_15-51-12(ESB-GTACR2)"             
 [50] "2023-07-12_14-25-09_e36_roi_11-IDOC_roi_NA"  
 [51] "2023-07-12_14-25-09_e36_roi_12-IDOC_roi_NA"  
 [52] "2023-07-12_14-25-09_e36_roi_13-IDOC_roi_NA"  
 [53] "2023-07-12_14-25-09_e36_roi_14-IDOC_roi_NA"  
 [54] "2023-07-12_14-25-09_e36_roi_15-IDOC_roi_NA"  
 [55] "2023-07-12_14-25-09_e36_roi_16-IDOC_roi_NA"  
 [56] "2023-07-12_14-25-09_e36_roi_17-IDOC_roi_NA"  
 [57] "2023-07-12_14-25-09_e36_roi_18-IDOC_roi_NA"  
 [58] "2023-07-12_14-25-09_e36_roi_19-IDOC_roi_NA"  
 [59] "2023-07-12_14-25-09(ET_spaced_overnight)"    
 [60] "2023-07-13_16-05-32_e36_roi_11-IDOC_roi_NA"  
 [61] "2023-07-13_16-05-32_e36_roi_12-IDOC_roi_NA"  
 [62] "2023-07-13_16-05-32_e36_roi_13-IDOC_roi_NA"  
 [63] "2023-07-13_16-05-32_e36_roi_14-IDOC_roi_NA"  
 [64] "2023-07-13_16-05-32_e36_roi_15-IDOC_roi_NA"  
 [65] "2023-07-13_16-05-32_e36_roi_16-IDOC_roi_NA"  
 [66] "2023-07-13_16-05-32_e36_roi_17-IDOC_roi_NA"  
 [67] "2023-07-13_16-05-32(ET_spaced_overnight)"    
 [68] "2023-07-17_15-19-38(ET_3h)"                  
 [69] "2023-07-17_16-23-00(ET_1h)"                  
 [70] "2023-07-18_14-44-14(ET_3h)"                  
 [71] "2023-07-18_15-25-46_e42_roi_11-IDOC_roi_NA"  
 [72] "2023-07-18_15-25-46_e42_roi_12-IDOC_roi_NA"  
 [73] "2023-07-18_15-25-46_e42_roi_13-IDOC_roi_NA"  
 [74] "2023-07-18_15-25-46_e42_roi_14-IDOC_roi_NA"  
 [75] "2023-07-18_15-25-46_e42_roi_15-IDOC_roi_NA"  
 [76] "2023-07-18_15-25-46_

In [375]:
result_dir <- "IDOC_dataset/"
result_dir <- "/idoc_data/IDOC"
metadata_linked <- link_idoc_metadata(metadata, result_dir=result_dir, verbose=FALSE)
n_missing_animals <- nrow(metadata_linked[is.na(idoc_folder),])
if (n_missing_animals>0) {
    warning(paste(n_missing_animals, "animals could not be linked"))
}
metadata_linked <- metadata_linked[!is.na(idoc_folder),]
metadata_linked[, PRE_manual := PRE]
metadata_linked[, POST_manual := POST]

In [376]:
data.table::fwrite(x = metadata_linked, file = "IDOC_paper_metadata.csv")

In [377]:
# use raw csv
data_raw <- load_idoc_data(
    metadata_linked,
    # how many exits need to happen for the trial to be considered?
    min_exits=3,
    # load from trial 1 and trial 2, applying the min_exit requirement to each separately
    trials=1:2,
    use_incomplete_tests = USE_INCOMPLETE_TESTS,
    mm_decision_zone = 7,
    verbose=FALSE,
    # summary_FUN=average_trial
    summary_FUN=list(
        PRE=average_trial,
        POST=average_trial
    )
)

In [378]:
data_raw[substr(Files, 1, 10)=="2024-12-12", .(PRE_ROI, PRE, POST)]

PRE_ROI,PRE,POST
<int>,<dbl>,<dbl>
1,-0.16666667,0.26190476
2,-0.05555556,0.00000000
3,0.05555556,NA
4,-0.05555556,0.00000000
5,0.07142857,-0.07142857
6,-0.15000000,0.00000000
7,0.05555556,NA
8,0.05555556,-0.80000000
9,0.01587302,-0.14285714


In [379]:
index <- data_raw[PRE_ROI!="NONE", .SD[, .(.N, PRE_files, POST_files)], by=idoc_folder][, .SD[1,], by=idoc_folder]
index <- index[
    sapply(index$PRE_files, function(row) {all(!sapply(row, is.na))})
]
index <- index[
    sapply(index$POST_files, function(row) {all(!sapply(row, is.na))}),
]
data.table::fwrite(x=index, file="index.csv")

In [380]:
data_raw[,PRE_machine := PRE]
data_raw[,PRE_1_machine := PRE_1]
data_raw[,PRE_2_machine := PRE_2]
data_raw[,POST_machine := POST]
data_raw[,POST_1_machine := POST_1]
data_raw[,POST_2_machine := POST_2]

In [381]:
if (SCORING_SYSTEM=="manual") {
  message("Using human made scores")   
  data_raw[ , PRE := PRE_manual]
  data_raw[ , POST := POST_manual]
  data_raw[, PRE_criteria := "manual"]
  data_raw[, POST_criteria := "manual"]
    
} else if (SCORING_SYSTEM=="flexible") {
  message("Using flexible scores")   
  data_raw[ , PRE := PRE_machine]
  data_raw[ , POST := POST_machine]
  data_raw[, PRE_criteria := "machine"]
  data_raw[, POST_criteria := "machine"]
  data_raw[!is.na(PRE_manual) & !(PRE_Reason%in%c("", "?", "Machine-override")), PRE_criteria := "manual"]
  data_raw[!is.na(PRE_manual) & !(PRE_Reason%in%c("", "?", "Machine-override")), PRE := PRE_manual]
  data_raw[!is.na(POST_manual) & !(POST_Reason%in%c("", "?", "Machine-override")), POST_criteria := "manual"]
  data_raw[!is.na(POST_manual) & !(POST_Reason%in%c("", "?", "Machine-override")), POST := POST_manual]
    
} else {
  message("Using machine made scores")   
  data_raw[ , PRE := PRE_machine]
  data_raw[ , POST := POST_machine]
}

Using flexible scores



In [382]:
table(data_raw$POST_Reason)


                                               
                                          1221 
                                             ? 
                                             3 
                                  AOJ-override 
                                             3 
                discard due to LTM not working 
                                            20 
                    discard due to LTM working 
                                            16 
                                Human-override 
                                            43 
                       Human-override; discard 
                                             3 
Human-override; discard due to LTM not working 
                                             1 
                 IDOC2 not working as expected 
                                            31 
                                     Left-bias 
                                             7 
                              Machine-o

In [383]:
data.table::fwrite(x = data_raw, file = "raw_data_wide.csv")

In [384]:
data <- data.table::copy(data_raw)
data[is.na(POST), POST_Reason := "POST_NA"]
data[is.na(PRE), PRE_Reason := "PRE_NA"]
data[PRE > 0.2, PRE_Reason := "Pre-attraction"]
data[PRE < -0.2, PRE_Reason := "Pre-aversion"]


In [385]:
# data[Training=="6X_Massed", .(Files, PRE, POST)]

In [386]:
table(data$POST_Reason)


                                                            ? 
                           855                              3 
                  AOJ-override discard due to LTM not working 
                             3                             16 
    discard due to LTM working                 Human-override 
                            12                             39 
       Human-override; discard  IDOC2 not working as expected 
                             2                             17 
                     Left-bias               Machine-override 
                             4                             82 
                       POST_NA 
                           399 

In [387]:
table(data$PRE_Reason)


                                ?   Human-override Machine-override 
            1093                3                2               18 
          PRE_NA   Pre-attraction     Pre-aversion       Right-bias 
             108              144               63                1 

In [388]:
nrow(data[PRE_Reason%in%c("", "?") & POST_Reason %in% c("", "?", "Human-override", "Machine-override"),])

[1] 802

In [389]:
# verify there are not repeats (the same fly represent more than once)
counts <- data[, .N, by=fly_name_reference]
stopifnot(all(counts$N==1))

In [390]:
counts[N!=1,]

fly_name_reference,N
<chr>,<int>


In [391]:
data[, id := 1:.N]

In [392]:
data_long <- melt(data = data, measure.vars = c("PRE", "POST"), value.name = "PI", variable.name = "test")
data.table::fwrite(x = data_long, file = "tidy_data.csv")
data.table::fwrite(x = data, file = "tidy_data_wide.csv")

In [393]:
data[, .N, by=.(Genotype, Files)]

Genotype,Files,N
<chr>,<chr>,<int>
MB010B.(II)SPARC-Chrimson ISO,2021-09-30 (CH),12
MB010B.(II)SPARC-Chrimson ISO,2021-09-30 (CH1),9
MB010B.(II)SPARC-Chrimson ISO,2021-10-01 (CH),13
MB010B.(II)SPARC-Chrimson ISO,2021-10-01 (CH1),13
MB010B.(II)SPARC-Chrimson ISO,2021-10-01 (CH2),11
MB010B.(II)SPARC-GFP ISO,2021-10-02 (CH),13
MB010B.(II)SPARC-GFP ISO,2021-10-02 (CH1),13
MB010B.(II)SPARC-Chrimson ISO,2021-10-03 (CH),13
MB010B.(II)SPARC-Chrimson ISO,2021-10-03 (CH1),11


In [394]:
data_raw[POST_criteria=="manual", .N, by=POST_Reason]

POST_Reason,N
<chr>,<int>
Human-override,33
AOJ-override,3
discard due to LTM not working,7
discard due to LTM working,11


In [395]:
unique(data_raw$Files)

[1] "2021-09-30 (CH)"                             
  [2] "2021-09-30 (CH1)"                            
  [3] "2021-10-01 (CH)"                             
  [4] "2021-10-01 (CH1)"                            
  [5] "2021-10-01 (CH2)"                            
  [6] "2021-10-02 (CH)"                             
  [7] "2021-10-02 (CH1)"                            
  [8] "2021-10-03 (CH)"                             
  [9] "2021-10-03 (CH1)"                            
 [10] "2021-10-03 (CH2)"                            
 [11] "2021-10-04 (CH)"                             
 [12] "2021-10-04 (CH1)"                            
 [13] "2021-10-05 (CH)"                             
 [14] "2021-10-06 (CH)"                             
 [15] "2021-10-06 (CH1)"                            
 [16] "2021-10-27 (CH)"                             
 [17] "2021-10-27 (CH1)"                            
 [18] "2021-10-29 (CH)"                             
 [19] "2021-11-01 (CH)"                             
 [20] "2021-11-01 (CH1)"                            
 [21] "2021-11-03 (CH1)"                            
 [22] "2021-11-04 (CH)"                             
 [23] "2021-11-08 (CH)"                             
 [24] "2021-11-17 (CH)"                             
 [25] "2021-11-17 (CH1)"                            
 [26] "2021-11-24 (CH)"                             
 [27] "2021-11-27 (CH1)"                            
 [28] "2021-11-28 (CH)"                             
 [29] "2021-11-30 (CH)"                             
 [30] "2021-12-02 (CH)"                             
 [31] "2021-12-03 (CH)"                             
 [32] "2021-12-05 (CH)"                             
 [33] "2021-12-08 (CH)"                             
 [34] "2021-12-17 (CH)"                             
 [35] "2021-12-22 (CH)"                             
 [36] "2021-12-22 (CH1)"                            
 [37] "2021-12-28 (CH)"                             
 [38] "2022-01-03 (CH)"                             
 [39] "2022-06-23 (CH)"                             
 [40] "2022-06-23 (CH1)"                            
 [41] "2023-06-27_16-34-26(ET_3h)"                  
 [42] "2023-06-28_15-17-27(ET_3h)"                  
 [43] "2023-07-04_13-45-39(ET_3h)"                  
 [44] "2023-07-10_15-09-51(ET_3h)"                  
 [45] "2023-07-10_16-10-48(ET_1h)"                  
 [46] "2023-07-12_14-25-09(ET_spaced_overnight)"    
 [47] "2023-07-13_16-05-32(ET_spaced_overnight)"    
 [48] "2023-07-17_15-19-38(ET_3h)"                  
 [49] "2023-07-17_16-23-00(ET_1h)"                  
 [50] "2023-07-18_14-44-14(ET_3h)"                  
 [51] "2023-07-18_15-25-46(ET_LTM_spaced)"          
 [52] "2023-07-19_14-05-58(ET_3h)"                  
 [53] "2023-07-19_15-04-46(ET_1h)"                  
 [54] "2023-07-23_16-20-43(ESB-massed)"             
 [55] "2023-07-24_16-19-01(ESB-massed)"             
 [56] "2023-07-31_15-16-30(ET_LTM_spaced)"          
 [57] "2023-08-01_14-36-35(ET_LTM_spaced_SD)"       
 [58] "2023-08-09_13-56-17(ET_LTM_SD)"              
 [59] "2023-08-16_14-15-36(ET_LTM_SD)"              
 [60] "2023-08-17_14-45-53(ET_LTM_SD)"              
 [61] "2023-08-21_15-02-37(ET_LTM_SD)"              
 [62] "2023-08-22_14-26-35(ET_LTM_SD)"              
 [63] "2023-08-23_15-02-03(ET_LTM_SD)"              
 [64] "2023-08-24_14-58-09(ET_LTM_SD)"              
 [65] "2023-08-29_16-12-49(ET_LTM_SD)"              
 [66] "2023-08-30_16-21-50(ET_LTM_SD)"              
 [67] "2023-08-31_16-04-36(ET_LTM_SD)"              
 [68] "2023-09-05_15-42-19(ET_LTM_SD)"              
 [69] "2023-09-06_15-14-13(ET_LTM_SD)"              
 [70] "2023-09-07_15-47-15(ET_LTM_SD)"              
 [71] "2023-09-11_16-06-53(ET_LTM_SD)"              
 [72] "2023-09-12_15-08-10(ET_1hr_STM)"             
 [73] "2023-09-14_15-13-25_AOJ_6XMASSED_LTM"        
 [74] "2023-09-15_16-05-56(ET_LTM_SD)"              
 [75] "2023-09-21_13-50-21(ET_LTM_SD)"              
 [76] "2023-09-22_14-48-18(

In [396]:
unique(data_raw$US)

[1] "75V 0Ω"    ""          "75V 4.6kΩ" "75V 4.6Ω"